In [1]:
## Airbnb data visualization nd application of SVD

In [2]:
# https://towardsdatascience.com/airbnb-rental-listings-dataset-mining-f972ed08ddec
# http://insideairbnb.com/get-the-data.html

In [3]:
import pandas as pd
import numpy as np
from fitparse import FitFile
import os


from collections import defaultdict 

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_daq as daq
from jupyter_dash import JupyterDash
import plotly.graph_objs as go
import dash_table
from dash_table.Format import Format, Group, Scheme
import dash_table.FormatTemplate as FormatTemplate

import math
import folium

from datetime import datetime as dt


In [4]:
%load_ext watermark
%watermark -v -m -p pandas,nltk,wordcloud,numpy,plotly,dash,jupyter_dash,folium,collections,dash_core_components,dash_table,dash_html_components,datetime


CPython 3.7.7
IPython 7.19.0

pandas 1.1.0
nltk 3.5
wordcloud 1.8.1
numpy 1.19.1
plotly 4.12.0
dash 1.17.0
jupyter_dash 0.3.1
folium 0.11.0
collections unknown
dash_core_components 1.13.0
dash_table 4.11.0
dash_html_components 1.1.1
datetime unknown

compiler   : MSC v.1900 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores  : 8
interpreter: 64bit


Part 4 Folder Structure
Within the root folder, Dash will look for:
 1. “data” subfolder: containing the underlying data of the dashboard
 2. “assets” subfolder: containing additional files, like images, css files

In [ ]:
#Import the backend manipilating functions
from callbacks_amenities import *


####################################################################################################
# DEFINE THE APP
####################################################################################################

# Create the app for Jupyter notebook
app = JupyterDash(__name__)
server = app.server

# # This is where the main program runs It contains all the callbacks and also the helper functions for the callbacks
from layout_amenities import *

####################################################################################################################
#          This sets the layout id. For every page URL, this can be updated by display_page callback
####################################################################################################################

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])


####################################################################################################################
#           HELPER FUNCTIONS FOR THE CALLBACK- These have to be in the same cell or .py file, if one is created
#                                               as the callbacks
####################################################################################################################    
    
  
    

################################################################################################
#           Callbacks for Individual Rental maps
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='indv_listing_map',  component_property='srcDoc'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_chart(nbd):
    #Filter by neighborhood
  
    filename =  saveListingMapHTML(nbd)
    return  open(filename, 'r').read()
    
################################################################################################
#           Callbacks for Aggregate for Neighborhood Rental maps
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='nbd_map', component_property='srcDoc'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_chart(nbd):
    #Filter by neighborhood
  
    filename = saveNeighborhoodMapHTML(nbd)
    return  open(filename, 'r').read()
    
################################################################################################
#           Callback for total card
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='total_nbd', component_property='children'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_chart(nbd):
    return f"{getNumNbds(nbd)}\nNumber of neighborhoods"      
    
   
    
################################################################################################
#           Callback for total card
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='total_listings', component_property='children'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_chart(nbd):
    return f"{getAvailableListingsNum(nbd)}\nNumber of Listings"      
 

    
################################################################################################
#           Callback for neighborhood name card
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='nbd_name', component_property='children'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_chart(nbd):
    if nbd == 'All':
        return f"Seattle"
    else:
        return f"{nbd}"      
    
        
################################################################################################
#           Callback for average price
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='avg_price', component_property='children'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_chart(nbd):
    return f"USD {getAvgPriceNbd(nbd)}\nAverage rent"       


################################################################################################
#           Callback for Sankey chart
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='sankey_chart', component_property='figure'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_sankeychart(nbd):
    fig = createSankeyChart(nbd)
    return fig  




################################################################################################
#           Callback for horizontal area-property chart
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='areaprop_chart', component_property='figure'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value')
       
)
def update_areapropchart(nbd):
    fig = createAreaPropTypeMap(nbd)
    return fig  



################################################################################################
#           Callback for vertical amenities frequency chart
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='amenitiesbar_chart', component_property='figure'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value'),
       dash.dependencies.Input(component_id='freq_slider', component_property='value')
       
)
def update_amenitiesbarchart(nbd, topn):
    fig = createAmenitiesFreqMap(nbd, topn)
    return fig  



################################################################################################
#           Callback for vertical amenities frequency chart
################################################################################################
@app.callback(
       dash.dependencies.Output(component_id='wordcloud_chart', component_property='src'),
       dash.dependencies.Input(component_id='nbd_dropdown', component_property='value'),
       dash.dependencies.Input(component_id='freq_slider', component_property='value')
       
)
def update_wordcloudimage(nbd, topn):
    src = createAmenitiesWordCloud(nbd, topn)
    return src


#################################################################################################
#          URL Page Router
################################################################################################


@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/apps/rental-main':
         return rental
    elif pathname == '/apps/rentaldetails':
         return rentaldetails
    else:
        return rental # This is the "home page"
    
    
    
################################################################################################
#       Submit the application to the server
################################################################################################

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /assets/001_dosis.css?m=1605042111.9810834 HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /assets/003_dashboard-style.css?m=1607890465.4238274 HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_3m1604000066.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_3m1604000066.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_3m1604000066.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_3m1604000066.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:03] "GET /assets/002_bootstrap-4.4.1.css?m=1605042129.6948204 HTTP/1.1" 200 -
127.0.0.

            amenity_token  freq
0                      tv  4768
1            free parking  4306
2                    wifi  4266
3             smoke alarm  4245
4                 heating  3911
5                 shampoo  3911
6              hair dryer  3902
7                 hangers  3876
8                    iron  3789
9   carbon monoxide alarm  3765
10                kitchen  3751
11     friendly workspace  3561
12                 washer  3402
13                  dryer  3389
14              hot water  3336
15  dishes and silverware  3204
16           refrigerator  3174
17           coffee maker  3079
18              microwave  3038
19      fire extinguisher  2895
            amenity_token  freq
0                      tv  4768
1            free parking  4306
2                    wifi  4266
3             smoke alarm  4245
4                 heating  3911
5                 shampoo  3911
6              hair dryer  3902
7                 hangers  3876
8                    iron  3789
9   carb

127.0.0.1 - - [02/Feb/2021 19:02:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:11] "GET /assets/amenities_all_20.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:30] "POST /_dash-update-component HTTP/1.1" 200 -


                 amenity_token  freq
0                           tv  4768
1                 free parking  4306
2                         wifi  4266
3                  smoke alarm  4245
4                      heating  3911
5                      shampoo  3911
6                   hair dryer  3902
7                      hangers  3876
8                         iron  3789
9        carbon monoxide alarm  3765
10                     kitchen  3751
11          friendly workspace  3561
12                      washer  3402
13                       dryer  3389
14                   hot water  3336
15       dishes and silverware  3204
16                refrigerator  3174
17                coffee maker  3079
18                   microwave  3038
19           fire extinguisher  2895
20              cooking basics  2681
21                  bed linens  2601
22                       stove  2586
23                        oven  2549
24            private entrance  2535
25  extra pillows and blankets  2200
2

...

127.0.0.1 - - [02/Feb/2021 19:02:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:43] "GET /assets/amenities_beacon%20hill_30.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:46] "POST /_dash-update-component HTTP/1.1" 200 -


                 amenity_token  freq
0                           tv   241
1                  smoke alarm   187
2                         wifi   186
3                      hangers   181
4                      kitchen   181
5        carbon monoxide alarm   181
6                   hair dryer   179
7                      shampoo   178
8                         iron   177
9                       washer   165
10                       dryer   165
11                coffee maker   162
12                refrigerator   160
13                   microwave   159
14          friendly workspace   158
15                        oven   155
16                     heating   154
17                  bed linens   150
18                       stove   145
19       dishes and silverware   141
20     long term stays allowed   140
21              cooking basics   139
22                   hot water   138
23  extra pillows and blankets   122
24                  dishwasher   119
25                         gym   111
2

127.0.0.1 - - [02/Feb/2021 19:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:47] "GET /assets/amenities_cascade_30.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:55] "POST /_dash-update-component HTTP/1.1" 200 -


                 amenity_token  freq
0                           tv  4768
1                 free parking  4306
2                         wifi  4266
3                  smoke alarm  4245
4                      heating  3911
5                      shampoo  3911
6                   hair dryer  3902
7                      hangers  3876
8                         iron  3789
9        carbon monoxide alarm  3765
10                     kitchen  3751
11          friendly workspace  3561
12                      washer  3402
13                       dryer  3389
14                   hot water  3336
15       dishes and silverware  3204
16                refrigerator  3174
17                coffee maker  3079
18                   microwave  3038
19           fire extinguisher  2895
20              cooking basics  2681
21                  bed linens  2601
22                       stove  2586
23                        oven  2549
24            private entrance  2535
25  extra pillows and blankets  2200
2

127.0.0.1 - - [02/Feb/2021 19:02:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2021 19:02:55] "GET /assets/amenities_all_30.png HTTP/1.1" 200 -
